In [3]:
import docIterators
import pymongo
from pymongo import MongoClient
import numpy as np
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE, MDS
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import collections
import codecs
import json
import random
from bhtsne.bhtsne import TSNE as BHTSNE
import sys
import analysis_tools as at

In [4]:
conn=['mongodb://localhost:27017','Cherry','Passionfruit']
#iterator = docIterators.StrawberryIter(conn,iter_type='VECTORS')
#mc = MongoClient(conn[0])[conn[1]][conn[2]]

In [40]:
with codecs.open('journals.json','r',encoding='utf8') as f:
    c=json.load(f)
c=collections.Counter(c)
vector_type='d2v-d'
journo_coll={}
for journal in zip(*c.most_common()[:20])[0]:
    journo_vectors=[]
    curs = mc.find({'journal':journal})
    for rec in curs[:1000]:
        vectors = rec['vectors']
        journo_vectors.append(vectors[vector_type])
    journo_coll[journal]=journo_vectors
    print(journal + ' collected')

Chemistry - A European Journalcollected
Chem. Lett.collected
Phys. Chem. Chem. Phys.collected
Journal of Materials Chemistrycollected
Food Chemistrycollected
Bioorganic & Medicinal Chemistry Letterscollected
Journal of Heterocyclic Chemistrycollected
Chemistry of Heterocyclic Compoundscollected
Int. J. Quantum Chem.collected
ChemInformcollected
J. Polym. Sci. A Polym. Chem.collected
Journal of Radioanalytical and Nuclear Chemistrycollected
European Journal of Organic Chemistrycollected
Bioorganic & Medicinal Chemistrycollected
Anal Bioanal Chemcollected
Eur. J. Inorg. Chem.collected
Journal of Solid State Chemistrycollected
Organic & Biomolecular Chemistrycollected
Environmental Toxicology and Chemistrycollected
Materials Chemistry and Physicscollected


In [42]:
vecs= [a for b in journo_coll.values() for a in b]
#t = TSNE(init='pca',n_components=2)
t=BHTSNE(n_components=2)
reduced_vecs = t.fit_transform(vecs)
colors=iter(plt.cm.rainbow(np.linspace(0,1,len(journo_coll.keys()))))
start_ind=0
for j in journo_coll.keys():
    no_to_plot=len(journo_coll[j])
    end_ind=start_ind+no_to_plot
    color=colors.next()
    plt.plot(reduced_vecs[start_ind:end_ind,0], reduced_vecs[start_ind:end_ind,1], '.',c=color)
    start_ind=end_ind
    plt.show()

In [13]:
vecs= []
ind=0
for record in iterator:
    vecs.append(record['vectors']['d2v-d'])
    if ind==50000:
        break
    ind+=1
#t = TSNE(init='pca',n_components=2)
t=BHTSNE(n_components=2)
reduced_vecs = t.fit_transform(vecs)
plt.plot(reduced_vecs[:,0], reduced_vecs[:,1],'o')
plt.show()

[#         ] 10% 50000bhtsne/bh_tsne


KeyboardInterrupt: 

In [5]:
iterator = docIterators.StrawberryIter(conn,iter_type='VECTORS',query={'source_url':{'$regex':'www.ch.cam.ac.uk'}})
vec,dois=at.get_vectors(iterator)
mat = at.cosine_mat(vec,vec)

In [7]:
mat = at.cosine_mat(vec,vec)

In [8]:
def csv_gephi_exporter(dois,mat,filename='cam_connect.csv',thresh=0.35):
    with open(filename,'w') as f:
        masked=mat*(mat>thresh)
        f.write(';'+';'.join(dois)+'\n')
        for i in range(mat.shape[0]):
            masked[i,i]=0.
            ex=(';'.join([str(j) for j in masked[i] if j>thresh]))
            ex=dois[i]+';'+ex+'\n'
            f.write(ex)


In [35]:
def gexf_gephi_exporter(dois,mat,filename='cam_connect2.gexf',thresh=.35):
    node_count=len(dois)
    edge_count=(np.sum(mat>thresh)-mat.shape[0])/2
    header='''<?xml version="1.0" encoding="UTF-8"?>
<gexf xmlns:viz="http:///www.gexf.net/1.1draft/viz" version="1.1" xmlns="http://www.gexf.net/1.1draft">
<meta lastmodifieddate="2010-03-03+23:44">
<creator>Gephi 0.7</creator>
</meta>
<graph defaultedgetype="undirected" idtype="string" type="static">
<nodes count="'''
    header=header+str(node_count)+'">'
    footer='''\n</edges>\n</graph>\n</gexf>'''
    with open(filename,'w') as f:
        f.write(header)
        for i in range(node_count):
            node='\n<node id="'+str(float(i))+'" label="'+dois[i]+'"/>'
            f.write(node)
        f.write('\n</nodes>\n<edges count="'+str(edge_count)+'">')
        edge_no=0
        for i in range(node_count):
            for j in range(i+1,node_count):
                if mat[i,j]>thresh:
                    f.write(
                        '\n<edge id="'+
                        str(float(edge_no))+
                        '" source="'+str(float(i))
                        +'" target="'+str(float(j))+
                        '" weight="'+str(mat[i,j])+'"/>')
                    edge_no+=1
        f.write(footer)

In [36]:
gexf_gephi_exporter(dois,mat)

In [249]:
counts, edges = np.histogram(mat,bins=100)
plt.plot(centres,list(counts))
plt.show()

In [239]:
plt.show()

In [53]:
def get_classes(csv_file_name):
    doi_dict={}
    with codecs.open(csv_file_name) as csvf:
        reader = csv.DictReader(csvf,delimiter=',')
        for row in reader:
            doi_dict[row['label']]=row['modularity_class']
    c = collections.Counter(doi_dict.values())
    classes_list={}
    for clas in c.keys():
        classes_list[clas]=[k for k,v in classes.items() if v==clas]
    return doi_dict, classes_list

In [54]:
a,b=get_classes('camConnect2Regions.csv')

In [57]:
b['10']

['10.1021/jo102474u',
 '10.1039/b704786j',
 '10.1021/jo8022395',
 '10.1021/acs.orglett.5b02769',
 '10.1039/a804268c',
 '10.1039/c1cs15171a',
 '10.1002/anie.200390265',
 '10.1002/anie.201501995',
 '10.1021/jo00193a014',
 '10.1039/c19670000406',
 '10.1002/anie.201409356',
 '10.1021/ol9024259',
 '10.1039/c4fd00162a',
 '10.1039/b606882k',
 '10.1039/p19930002893',
 '10.1081/scc-120015805',
 '10.1039/cs9811000083',
 '10.1039/c2cc15453f',
 '10.1080/01418619808214238',
 '10.1021/ol801399w',
 '10.1002/adma.200500113',
 '10.1039/b003789n',
 '10.1002/ardp.19923250407',
 '10.1021/ol100693c',
 '10.1021/ol035848h',
 '10.1039/a700655a',
 '10.1021/bi00270a022',
 '10.1039/p19900000617',
 '10.1039/p19770001131',
 '10.1021/bi00294a029',
 '10.1039/c39860000344',
 '10.1002/chem.201500536',
 '10.1002/anie.200705566',
 '10.1021/ol4027107',
 '10.1039/b502213d',
 '10.1021/ja401840j',
 '10.1039/p19960001171',
 '10.1002/tcr.201100041',
 '10.1002/chem.200700302',
 '10.1002/anie.200502294',
 '10.1021/ol006766l',
 

In [64]:
iterator = docIterators.StrawberryIter(conn,iter_type='VECTORS',from_list=b['10'])
vec,dois=at.get_vectors(iterator)
mat = at.cosine_mat(vec,vec)

[######### ] 99% 1380



In [63]:
reload(docIterators)

<module 'docIterators' from 'docIterators.py'>

In [66]:
counts, edges = np.histogram(mat,bins=100)
centres = [(edges[i]+edges[i+1])/2 for i in range(100)]
plt.plot(centres,list(counts))
plt.show()

In [69]:
gexf_gephi_exporter(b['10'],mat,filename='10cluster.gexf')

In [68]:
def gexf_gephi_exporter(dois,mat,filename='cam_connect2.gexf',thresh=.35):
    node_count=len(dois)
    edge_count=(np.sum(mat>thresh)-mat.shape[0])/2
    header='''<?xml version="1.0" encoding="UTF-8"?>
<gexf xmlns:viz="http:///www.gexf.net/1.1draft/viz" version="1.1" xmlns="http://www.gexf.net/1.1draft">
<meta lastmodifieddate="2010-03-03+23:44">
<creator>Gephi 0.7</creator>
</meta>
<graph defaultedgetype="undirected" idtype="string" type="static">
<nodes count="'''
    header=header+str(node_count)+'">'
    footer='''\n</edges>\n</graph>\n</gexf>'''
    with open(filename,'w') as f:
        f.write(header)
        for i in range(node_count):
            node='\n<node id="'+str(float(i))+'" label="'+dois[i]+'"/>'
            f.write(node)
        f.write('\n</nodes>\n<edges count="'+str(edge_count)+'">')
        edge_no=0
        for i in range(node_count):
            for j in range(i+1,node_count):
                if mat[i,j]>thresh:
                    ex = (mat[i,j]-thresh)
                    f.write(
                        '\n<edge id="'+
                        str(float(edge_no))+
                        '" source="'+str(float(i))
                        +'" target="'+str(float(j))+
                        '" weight="'+str(ex)+'"/>')
                    edge_no+=1
        f.write(footer)